In [1]:
# Import the needed Packages
# FMU Simulation
import MoBASimulator as mb
# Numpy
import numpy as np
# Pandas for storage
import pandas as pd
# Algorithms
import Algorithms as alg
from Algorithms import TUBScolorscale

In [ ]:
result = pd.DataFrame(columns=('T_A', 'Q_C', 'K11', 'K12', 'K21', 'K22', 'T11', 'T12', 'T21', 'T22', 'L11', 'L12', 'L21', 'L22'))

# Load a Model
sim = mb.Simulator()
sim.clear()
sim.loadModel("C:/Users/juliu/Documents/Thesis/Modelica/FMU/2_2_Physical/System_External.fmu")
# Show log window
sim.showLogWindow()

# Set the simulation Parameter
sim_time = 6e4
T_amb = np.linspace(273.15,303,5) # Ambient Temperatures in K
Q_C = -70e3*np.ones_like(T_amb) # Heats in kJ/s
stepTime = 5e4 # Time for the step
step_N = -.3 # height of the step for fan
step_A = 0.3 # Height of the step for valve

In [ ]:
for number in range(len(T_amb)):
    # First experiment Fan -> Temp / Pressure
    sim.reloadModel()
    # Set the Parameter
    sim.set({"fmu.T_A":T_amb[number], "fmu.Q_C":Q_C[number],
         "fmu.booleanStep.startTime":stepTime-1.0,
        "fmu.step_T.startTime": stepTime, "fmu.step_T.height": step_N,
        "fmu.step_P.startTime": sim_time, "fmu.step_P.height": 0.})
    #sim.showParameterDialog()
    # Simulate
    res_Fan = sim.simulate(1,sim_time)

    # Repeat for Valve -> Temp/Pressure
    # Physical : Valve -> Pressure is negat
    sim.reloadModel()
    # Set the Parameter
    sim.set({"fmu.T_A":T_amb[number], "fmu.Q_C":Q_C[number],
         "fmu.booleanStep.startTime":stepTime-1.0,
        "fmu.step_T.startTime": sim_time, "fmu.step_T.height": 0.,
        "fmu.step_P.startTime": stepTime, "fmu.step_P.height": step_A})
    # Simulate
    res_Valve = sim.simulate(1,sim_time)

    # Get the signals
    y_P = res_Fan["fmu.p_H"]
    u_P =  step_A*np.ones_like(y_P)
    y_T = res_Fan["fmu.T_H"]
    u_T =  step_N*np.ones_like(y_T)
    time = res_Fan["time"]

    # Rescale the signals for identification
    # Get Data after Step -> For the Fan \delta T approx 400 s.
    # Get Temperature data
    y_T_I = y_T[np.where((time>=stepTime) & (time < (stepTime+400.)))]
    y_T_I = y_T_I-y_T_I[0]
    u_T_I = u_T[np.where((time>=stepTime) & (time <(stepTime+400.)))]
    t_T_I = time[np.where((time>=stepTime) & (time < (stepTime+400.)))]-stepTime

    # Same for pressure
    y_P_I = y_P[np.where((time>=stepTime) & (time < stepTime+400.))]
    y_P_I = y_P_I-y_P_I[0] 
    u_P_I = u_P[np.where((time>=stepTime) & (time < stepTime+400.))]
    t_P_I = time[np.where((time>=stepTime) & (time < stepTime+400.))]-stepTime

    # The needed Parameter
    K = np.zeros((2,2))
    T = np.zeros((2,2))
    L = np.zeros((2,2))

    # Identify from Fan to Temp [0][0] and from Fan to Pressure[1][0]
    K[0][0],T[0][0],L[0][0]=alg.Integral_Identification(y_T_I,u_T_I,t_T_I)
    K[1][0],T[1][0],L[1][0]=alg.Integral_Identification(y_P_I,u_T_I,t_P_I)

    # Get the signals
    steptime = sim.get("fmu.booleanStep.startTime")
    y_P = res_Valve["fmu.p_H"]
    u_P =  step_A*np.ones_like(y_P)
    y_T = res_Valve["fmu.T_H"]
    u_T =  step_N*np.ones_like(y_T)
    time = res_Valve["time"]

    # Rescale the signals for identification
    # Get Data after Step
    y_T_I = y_T[np.where((time>=steptime) & (time < (steptime+600.)))]
    y_T_I = y_T_I-y_T_I[0]
    u_T_I = u_T[np.where((time>=steptime) & (time <(steptime+600.)))]
    t_T_I = time[np.where((time>=steptime) & (time < (steptime+600.)))]-steptime

    # Same for P
    y_P_I = y_P[np.where((time>=steptime) & (time < steptime+600.))]
    y_P_I = y_P_I-y_P_I[0]
    u_P_I = u_P[np.where((time>=steptime) & (time < steptime+600.))]
    t_P_I = time[np.where((time>=steptime) & (time < steptime+600.))]-steptime

    #Identify from Valve to Temp [0][1] and from Valve to Pressure[1][1]
    K[0][1],T[0][1],L[0][1]=alg.Integral_Identification(y_T_I,u_P_I,t_T_I)
    K[1][1],T[1][1],L[1][1]=alg.Integral_Identification(y_P_I,u_P_I,t_P_I)

    result.loc[number,:2] = [T_amb[number], Q_C[number]]
    result.loc[number,2:] = np.hstack((K.flatten(),np.hstack((T.flatten(),L.flatten())))).tolist()
    result.to_csv('PhysicalIdentification_90.csv', sep=';')
    print(result)

      T_A    Q_C     K11      K12      K21     K22      T11      T12      T21  \
0  273.15 -70000 -2.4785  4.47579 -1.30873 -3.1878  50.0487  45.7833  15.1501   

      T22      L11      L12     L21      L22  
0  37.103  22.5795  19.1258  9.3006  16.3005  
       T_A    Q_C      K11      K12       K21      K22      T11      T12  \
0   273.15 -70000  -2.4785  4.47579  -1.30873  -3.1878  50.0487  45.7833   
1  280.612 -70000 -1.00984  6.11555 -0.645238 -2.11424    48.52  44.6993   

       T21      T22      L11      L12      L21      L22  
0  15.1501   37.103  22.5795  19.1258   9.3006  16.3005  
1  18.5886  36.5468  13.9193  16.6197  7.65198  16.0777  

In [ ]:
result

In [7]:
# Make a controller
#KY, B, D = alg.Control_Decoupled(K,T,L,0.5*np.ones_like(K))
KY, B, D = alg.Control_Decentral(K,T,L)
#Reload Model
sim.reloadModel()
# Set the parameter for a controller simulation
params = {"fmu.step_P.startTime":2e3,"fmu.step_P.height":.0, # Input disturbance from Valve
          "fmu.step_T.startTime":5e3, "fmu.step_T.height":.0, # Input disturbance from Fan
          "fmu.booleanStep.startTime":10e5} # Never turn off the controller
for outputs in range(1,3):
    for inputs in range(1,3):
        params.update({"fmu.multivariable_Controller.KP["+str(outputs)+","+str(inputs)+"]": KY[outputs-1][inputs-1][0]})
        params.update({"fmu.multivariable_Controller.KI["+str(outputs)+","+str(inputs)+"]": KY[outputs-1][inputs-1][1]})
        params.update({"fmu.multivariable_Controller.D["+str(outputs)+","+str(inputs)+"]": D[outputs-1][inputs-1]})
        params.update({"fmu.multivariable_Controller.B["+str(outputs)+","+str(inputs)+"]": B[outputs-1][inputs-1]})
sim.set(params)
sim.showParameterDialog()
# Get a change in Dynamics
time_table = [0.0,5e3,7e3,9e3]
tamb_table = [T_amb, T_amb+5, T_amb+5,T_amb+10]
qc_table = [Q_C,Q_C,Q_C-3e3,Q_C-3e3]
sim.setInputTable({"time":time_table, "fmu.T_A":tamb_table, "fmu.Q_C":qc_table})

# Simulate the closed loop
res_Control = sim.simulate(1,10e3)

In [42]:
# Simulate the closed loop

res.keys()

['fmu.Physical_System.heatBoundaryWithInputs_MTCabinet_Einzeln.heatPort.Q_flow',
 'fmu.Physical_System.boundary.summary.T_outflow',
 'fmu.multivariable_Controller.u_m[1]',
 'fmu.Physical_System.yTGC',
 'fmu.Physical_System.Qdotkaelte',
 'fmu.Physical_System.boundary2.T_in',
 'fmu.multivariable_Controller.u_m[2]',
 'fmu.Physical_System.boundary.T_in',
 'fmu.Physical_System.vle_gas_aus.T',
 'fmu.Physical_System.realExpression.y',
 'fmu.Physical_System.heatBoundaryWithInputs_MTCabinet_Einzeln.Q_flow_in',
 'fmu.T_A',
 'fmu.Physical_System.gc_aus_T',
 'fmu.Q_C',
 'time',
 'fmu.T_H',
 'fmu.p_H',
 'fmu.Physical_System.yHP',
 'fmu.Physical_System.boundary2.summary.T_outflow',
 'fmu.Physical_System.TUmgebung',
 'fmu.Physical_System.realExpression3.y']

In [8]:
# Get the signals
y_T_D = res_Control["fmu.T_H"]
y_P_D = res_Control["fmu.p_H"]
n_T_D = res_Control["fmu.Physical_System.realExpression.y"]
A_P_D = res_Control["fmu.Physical_System.realExpression3.y"]
t_D = res_Control["time"]

p = bk.figure()
p.line(t_D,y_T_D)
bk.show(p)

In [66]:
#Simulate Further
sim2 = mb.Simulator()
sim2.clear()
sim2.loadModel("C:/Users/juliu/Documents/Thesis/Modelica/FMU/2_2_Physical/System_External.fmu")
sim2.showLogWindow()

In [67]:
sim2.set(params)
#Make a disturbance plot
sim2.set({"fmu.T_A":293.15, "fmu.Q_C":-60e3})
sim2.set({"fmu.step_T.height": -1, "fmu.step_T.startTime": 1000,"fmu.step_P.height": .5, "fmu.step_P.startTime": 1500})
sim2.set({"fmu.booleanStep.startTime":1e5})
sim2.showParameterDialog()

0

In [68]:
res2 = sim2.simulate(1,4e3)

In [69]:
y_T_D = res2["fmu.T_H"]
y_P_D = res2["fmu.p_H"]
t_D = res2["time"]

p = bk.figure()
p.line(t_D,y_T_D, color="red")
p.line(t_D,y_P_D)
bk.show(p)

In [71]:

# Make a controller
KY, B, D = alg.Control_Decoupled(K,T,L,10*np.eye(2), pairing=[0,1])

#Reload Model
sim2.reloadModel()
# Set the parameter
params = {}
for outputs in range(1,3):
    for inputs in range(1,3):
        params.update({"fmu.multivariable_Controller.KP["+str(outputs)+","+str(inputs)+"]": KY[outputs-1][inputs-1][0]})
        params.update({"fmu.multivariable_Controller.KI["+str(outputs)+","+str(inputs)+"]": KY[outputs-1][inputs-1][1]})
        params.update({"fmu.multivariable_Controller.D["+str(outputs)+","+str(inputs)+"]": D[outputs-1][inputs-1]})
        params.update({"fmu.multivariable_Controller.B["+str(outputs)+","+str(inputs)+"]": B[outputs-1][inputs-1]})
sim2.set(params)
print(KY,B,D)
#Make a disturbance plot
sim2.set({"fmu.T_A":293.15, "fmu.Q_C":-60e3})
sim2.set({"fmu.step_T.height": -1, "fmu.step_T.startTime": 1000,"fmu.step_P.height": .5, "fmu.step_P.startTime": 1500})
sim2.set({"fmu.booleanStep.startTime":1e5})
sim2.showParameterDialog()

Modified Detuning Iterationts 0
Modified Detuning Iterationts 0
(array([[[-0.62224787, -0.01546867, -0.        ],
        [ 0.        ,  0.        ,  0.        ]],

       [[ 0.        ,  0.        ,  0.        ],
        [-0.14967252, -0.00308461, -0.        ]]]), array([[ 0.34385894,  0.        ],
       [ 0.        ,  0.        ]]), array([[ 1.        ,  4.5341174 ],
       [-0.35912127,  1.        ]]))


0

In [72]:
res3 = sim2.simulate(1,4e3)

In [73]:
y_T_D2 = res3["fmu.T_H"]
y_P_D2 = res3["fmu.p_H"]
t_D2 = res3["time"]

p = bk.figure()
p.line(t_D,y_T_D, color="red")
p.line(t_D,y_P_D, color = "red")
p.line(t_D2,y_T_D2, color="blue")
p.line(t_D2,y_P_D2, color = "blue")
bk.show(p)

In [22]:
# Make a controller
KY, B, D = alg.Control_Decoupled(K,T,L,1*np.eye(2), pairing=[0,1])
D[0][0] = 0.5*D[0][0]
#Reload Model
sim2.reloadModel()
# Set the parameter
params = {}
for outputs in range(1,3):
    for inputs in range(1,3):
        params.update({"fmu.multivariable_Controller.KP["+str(outputs)+","+str(inputs)+"]": KY[outputs-1][inputs-1][0]})
        params.update({"fmu.multivariable_Controller.KI["+str(outputs)+","+str(inputs)+"]": KY[outputs-1][inputs-1][1]})
        params.update({"fmu.multivariable_Controller.D["+str(outputs)+","+str(inputs)+"]": D[outputs-1][inputs-1]})
        params.update({"fmu.multivariable_Controller.B["+str(outputs)+","+str(inputs)+"]": B[outputs-1][inputs-1]})
sim2.set(params)
print(KY,B,D)
#Make a disturbance plot
sim2.set({"fmu.T_A":293.15, "fmu.Q_C":-60e3})
sim2.set({"fmu.step_T.height": -1, "fmu.step_T.startTime": 1000,"fmu.step_P.height": .5, "fmu.step_P.startTime": 1500})
sim2.set({"fmu.booleanStep.startTime":1e5})
sim2.showParameterDialog()
sim2.showLogWindow()
res4 = sim2.simulate(1,4e3)

Modified Detuning Iterationts 0
Modified Detuning Iterationts 6


NameError: name 'sim2' is not defined

In [21]:

y_T_D3 = res4["fmu.T_H"]
y_P_D3 = res4["fmu.p_H"]
t_D3 = res4["time"]

p = bk.figure()
p.line(t_D,y_T_D, color="red")
p.line(t_D,y_P_D, color = "red")
p.line(t_D2,y_T_D2, color="blue")
p.line(t_D2,y_P_D2, color = "blue")
p.line(t_D3,y_T_D3, color="green")
p.line(t_D3,y_P_D3, color = "green")
bk.show(p)

NameError: name 'res4' is not defined

In [87]:
sim2.reloadModel()
#Make a disturbance plot
sim2.set({"fmu.T_A":300.00, "fmu.Q_C":-60e3})
sim2.set({"fmu.step_T.height": -1, "fmu.step_T.startTime": 5000,"fmu.step_P.height": .5, "fmu.step_P.startTime": 15000})
sim2.set({"fmu.booleanStep.startTime":5e5})
sim2.showLogWindow()
res4 = sim2.simulate(1,10e3)

In [88]:
y_T_D3 = res4["fmu.T_H"]
y_P_D3 = res4["fmu.p_H"]
t_D3 = res4["time"]

p = bk.figure()
p.line(t_D3,y_T_D3, color="green")
p.line(t_D3,y_P_D3, color = "green")
bk.show(p)